In [354]:
import rdflib
from rdflib import Graph, Literal, BNode, Namespace, RDF, URIRef
from rdflib.namespace import DC, FOAF
import spotlight
import urllib, json
import pandas as pd
from collections import Counter
import random

In [47]:
disease = URIRef('http://dbpedia.org/ontology/Disease')
drug = URIRef('http://dbpedia.org/ontology/Drug')
symp = URIRef('http://dbpedia.org/resource/Category:Symptoms_and_signs')
anat = URIRef('http://dbpedia.org/ontology/AnatomicalStructure')
medicin = URIRef('http://dbpedia.org/page/Medical_diagnosis')
therapy = URIRef('http://dbpedia.org/page/Therapy')
group = URIRef('http://dbpedia.org/page/Category:Human_development')

In [148]:
def spalq_condition_ds(con):
    g = rdflib.Graph()
    result = g.parse('cknns/'+con+'.ttl', format='turtle')
    #print (result)
    querystring = "SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {"
    querystring = querystring+"?d <"+RDF.type+"> <"+disease+"> . " 
    querystring = querystring+"?s <"+RDF.type+"> <"+symp+"> . }" 
    #print(querystring)
    qres = g.query(querystring)
    
    #print (qres)
    d = []
    s = []
    
    for row in qres:
        di = str(row.asdict()['d'].toPython()).split('/')[-1]
        if di not in d:
            d.append(di)
        si = str(row.asdict()['s'].toPython()).split('/')[-1]
        if si not in s:
            s.append(si)
    g.close()
    return d,s

In [149]:
def spalq_condition_pos(con):
    g = rdflib.Graph()
    result = g.parse('cknns/'+con+'.ttl', format='turtle')
    #print (result)
    querystring = "SELECT DISTINCT ?r WHERE {"
    querystring = querystring+"?r <"+RDF.type+"> <"+anat+"> . }" 
    #print(querystring)
    qres = g.query(querystring)
    
    #print (qres)
    r = []
    if not qres:
        return r
    else:
        for row in qres:
            ri = str(row.asdict()['r'].toPython()).split('/')[-1]
            if ri not in r:
                r.append(ri)
    g.close()
    return r

In [150]:
def spalq_condition_drug(con):
    g = rdflib.Graph()
    result = g.parse('cknns/'+con+'.ttl', format='turtle')
    #print (result)
    querystring = "SELECT DISTINCT ?r WHERE {"
    querystring = querystring+"?r <"+RDF.type+"> <"+drug+"> . }" 
    #print(querystring)
    qres = g.query(querystring)
    
    #print (qres)
    r = []
    if not qres:
        return r
    else:
        for row in qres:
            ri = str(row.asdict()['r'].toPython()).split('/')[-1]
            if ri not in r:
                r.append(ri)
    g.close()
    return r

In [151]:
def spalq_condition_treat(con):
    g = rdflib.Graph()
    result = g.parse('cknns/'+con+'.ttl', format='turtle')
    #print (result)
    querystring = "SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {"
    querystring = querystring+"?r <"+RDF.type+"> <"+therapy+"> . }" 
    #print(querystring)
    qres = g.query(querystring)
    
    #print (qres)
    r = []
    if not qres:
        return r
    else:
        for row in qres:
            ri = str(row.asdict()['r'].toPython()).split('/')[-1]
            if ri not in r:
                r.append(ri)
    g.close()
    return r

In [152]:
def spalq_condition_medic(con):
    g = rdflib.Graph()
    result = g.parse('cknns/'+con+'.ttl', format='turtle')
    #print (result)
    querystring = "SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {"
    querystring = querystring+"?r <"+RDF.type+"> <"+medicin+"> . }" 
    #print(querystring)
    qres = g.query(querystring)
    
    #print (qres)
    r = []
    if not qres:
        return r
    else:
        for row in qres:
            ri = str(row.asdict()['r'].toPython()).split('/')[-1]
            if ri not in r:
                r.append(ri)
    g.close()
    return r

In [159]:
def spalq_condition_people(con):
    g = rdflib.Graph()
    result = g.parse('cknns/'+con+'.ttl', format='turtle')
    #print (result)
    querystring = "SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {"
    querystring = querystring+"?r <"+RDF.type+"> <"+group+"> . }" 
    #print(querystring)
    qres = g.query(querystring)
    
    #print (qres)
    r = []
    if not qres:
        return r
    else:
        for row in qres:
            ri = str(row.asdict()['r'].toPython()).split('/')[-1]
            if ri not in r:
                r.append(ri)
    g.close()
    return r

In [349]:
Diseaselist = ['diabetes-mellitus','allergy','asthma','breast-cancer','chickenpox','flu','gallstone','heart-attack','hypertension','lung-cancer','pneumonia','urinary-incontinence','urinary-tract-infection','stroke','glaucoma']

In [358]:
def select_k(df_list):
    k=0
    if len(df_list)>4:
        k=3
    else:
        if 1<len(df_list)<4:
            k=2
        else:
            if len(df_list)>1:
                k=1
            else:
                k=0
    return k

In [360]:

p_df_test = pd.DataFrame(columns=['Condition','Disease', 'Symptom','Position','Group','Sex'])

i =0
for dis in Diseaselist:
    d,s = spalq_condition_ds(dis)
    v=select_k(d)
    dsampling = random.choices(d, k=v)
    v=select_k(s)
    ssampling = random.choices(s, k=v) 
    #print (d,s)
    #t = spalq_condition_treat(dis)
    #k=select_k(t)
    #tsampling = random.choices(t, k) 
    #print ('treat: ', t)
    p = spalq_condition_pos(dis)
    v=select_k(p)
    psampling = random.choices(p, k=v) 
    #print('position: ', p)
    #m = spalq_condition_medic(dis)
    #print('medic: ', m)
    #r = spalq_condition_drug(dis)
    #print('drug: ', r)
    gr = spalq_condition_people(dis)
    v=select_k(gr)
    grsampling = random.choices(gr, k=v)
    
    sex = ['Male','Female']
    sexsampling = random.choices(sex)
    #print('people: ', gr)
    #d,r,s,p,m,t,a = spalq_condition(dis)
    p_df_test.loc[i] = [dis,dsampling,ssampling,psampling,grsampling,sexsampling]
    i=i+1

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> .  does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http:/

SELECT DISTINCT ?r WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/AnatomicalStructure> . } does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/page/Category:Human_development does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/page/Category:Human_development does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/resource/Category:Symptoms_and_signs does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/resource/Category:Symptoms_and_signs> . } does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?r WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?r WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?r WHERE {?r <http://www.w3.org/1999/02/22-rdf-syn

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < does not look like a valid URI, trying to serialize this will break.
SELECT DISTI

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/page/Category:Human_development> . } does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k W

SELECT DISTINCT ?r WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/AnatomicalStructure does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?r WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/AnatomicalStructure> . } does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/page/Category:Human_development does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <ht

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/resource/Category:Symptoms_and_signs does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/resource/Category:Symptoms_and_signs> . } does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?r WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?r WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?r WHERE {?r <http://www.w3.org/1999/02/22-rdf-syn

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < does not look like a valid URI, trying to serialize this will break.
SELECT DISTI

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/page/Category:Human_development> . } does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k W

In [361]:
p_df_test

,Condition,Disease,Symptom,Position,Group,Sex
0,diabetes-mellitus,"[Disease, Renal_agenesis, diabetes-mellitus]","[Hyperglycemia, Polyphagia, Polyphagia]","[Velarization, Velarization]",[],[Female]
1,allergy,"[allergy, Allergic_rhinitis, Allergic_rhinitis]","[Genetic_disorder, Anaphylaxis, Erythema]","[Redox, Blood, Redox]",[],[Female]
2,asthma,"[Stenosis, Stenosis, Cough]","[Asthma, Asthma, Common_cold]","[Lung, Lung]","[Ageing, Ageing]",[Female]
3,breast-cancer,"[Cutaneous_condition, breast-cancer, Cutaneous...","[Breast_pain, Cancer, Breast_cancer]","[Breast, Breast, Experiment]",[],[Male]
4,chickenpox,"[Schizophrenia, Schizophrenia, Appetite]","[Fever, Blister, Itch]","[Experiment, Clothing, Blood]","[Ageing, Ageing]",[Male]
5,flu,"[Diarrhea, Sore_throat, Diarrhea]","[Pain, Pain, Otitis_media]","[Human_body, Human_body]",[],[Male]
6,gallstone,"[gallstone, Jaundice]","[Fever, Fever, Fever]","[Gallbladder, Pancreas]",[],[Male]
7,heart-attack,"[Splenic_injury, Schizophrenia, Medical_emerge...","[Pain, Myocardial_infarction, Coronary_artery_...","[Breathing, Breathing, Breathing]",[],[Male]
8,hypertension,"[Blood_pressure, Tinnitus, Visual_perception]","[Hypertension, Vertigo]","[Circulatory_system, Optic_nerve, Stomach]",[],[Male]
9,lung-cancer,"[Disease, Chronic_condition, Schizophrenia]","[Lung_cancer, Lung_cancer, Metastasis]","[Lung, Blood_cell, Cigar]",[],[Female]


In [377]:
cn = Namespace("http://healthchatbot.org/condtion/")

In [337]:
#prediction v1.0

In [450]:
def predict_ranking(symptom,pre_disease,position,agegroup,sex):
    p_df = pd.DataFrame(columns=['Condition','Disease', 'Symptom', 'Drug', 'Position','Medic','Treat', 'Group'])

    i =0
    for dis in Diseaselist:
        d,s = spalq_condition_ds(dis)
    #print (d,s)
        t = spalq_condition_treat(dis)
    #print ('treat: ', t)
        p = spalq_condition_pos(dis)
    #print('position: ', p)
        m = spalq_condition_medic(dis)
    #print('medic: ', m)
        r = spalq_condition_drug(dis)
    #print('drug: ', r)
        gr = spalq_condition_people(dis)
    #print('people: ', gr)
    #d,r,s,p,m,t,a = spalq_condition(dis)
        p_df.loc[i] = [dis,d,s,r,p,m,t,gr]
        i=i+1
    scoredict ={}
    matchdict = {}
    symtcount ={}
    i = 0 
    #print (agegroup)
    #symptom ranking
    sdg = rdflib.Graph()
    causes = URIRef(cn.causesDisease)
    anatomicOn = URIRef(cn.causeOnAnatomicStructure)
    drug = URIRef(cn.drugTo)
    diag = URIRef(cn.diagnosisTo)
    symptomOf = URIRef(cn.causesSymptom)
    treatment = URIRef(cn.treatmentTo)
    question=  URIRef(cn.question)
    answer = URIRef(cn.answerTo)
    anabout = URIRef(cn.aLabel)
    agegroup = URIRef(cn.causesAffectionToAgeGroup)
    probability = URIRef(cn.probablity)
    score = URIRef(cn.overallRankingScore)
    filepath='cknns/predictProbabilityGraph.ttl'
    for sym in p_df['Symptom']:
        avsym = 0
        for s in sym:
            for symi in symptom:
                if symi == s:
                    avsym = avsym+1
            symtcount[s]=avsym
            #print (s, avsym)
        for s in sym:
            if s in symptom:
                c = symtcount[s]
                #if(p_df['Condition'][i]=='urinary-incontinence'):
                    #print(s, p_df['Condition'][i])
                if p_df['Condition'][i] in scoredict:
                    x = scoredict.get(p_df['Condition'][i])
                    scoredict[p_df['Condition'][i]]=x+1/c
                    y = matchdict.get(p_df['Condition'][i])
                    matchdict[p_df['Condition'][i]]=y+1
                else:
                    scoredict[p_df['Condition'][i]]=1/c
                    matchdict[p_df['Condition'][i]]=1
                robj = URIRef('http://dbpedia.org/page/'+p_df['Condition'][i])
                subj = URIRef('http://dbpedia.org/page/'+s)
                sdg.add( (robj, RDF.type, disease) )
                sdg.add( (subj, RDF.type, symp) )
                sdg.add((robj, symptomOf, subj))
                sdg.add((robj, probability, Literal('symptom-'+s+':'+str(1/c))))
        i=i+1
    j = 0
    discount = {}
    for dis in p_df['Disease']:
        avsym = 0
        for d in dis:
            for disi in pre_disease:
                if disi == d and d != 'Disease':
                    avsym = avsym+1
            discount[d]=avsym
            #print (d, avsym)
        for d in dis:
            if d in pre_disease and d!='Disease':
                c = discount[d]
                #if(p_df['Condition'][j]=='urinary-incontinence'):
                    #print(d, p_df['Condition'][j])
                if p_df['Condition'][j] in scoredict:
                    x = scoredict.get(p_df['Condition'][j])
                    scoredict[p_df['Condition'][j]]=x+1/c
                    y = matchdict.get(p_df['Condition'][j])
                    matchdict[p_df['Condition'][j]]=y+1
                else:
                    scoredict[p_df['Condition'][j]]=1/c
                    matchdict[p_df['Condition'][j]]=1
                robj = URIRef('http://dbpedia.org/page/'+p_df['Condition'][j])
                subj = URIRef('http://dbpedia.org/page/'+d)
                sdg.add( (robj, RDF.type, disease) )
                sdg.add( (subj, RDF.type, disease) )
                sdg.add((robj, causes, subj))
                sdg.add((robj, probability, Literal('linkTo-'+d+':'+str(1/c))))
        j=j+1
    l = 0
    pstcount = {}
    for post in p_df['Position']:
        avsym = 0
        for pst in post:
            for pi in position:
                if pi == pst:
                    avsym = avsym+1
            pstcount[pst]=avsym
            #print (d, avsym)
        for pst in post:
            if pst in position:
                c = pstcount[pst]
                #if(p_df['Condition'][j]=='urinary-incontinence'):
                    #print(d, p_df['Condition'][j])
                if p_df['Condition'][l] in scoredict:
                    x = scoredict.get(p_df['Condition'][l])
                    scoredict[p_df['Condition'][l]]=x+1/c
                    y = matchdict.get(p_df['Condition'][l])
                    matchdict[p_df['Condition'][l]]=y+1
                else:
                    scoredict[p_df['Condition'][l]]=1/c
                    matchdict[p_df['Condition'][l]]=1
                robj = URIRef('http://dbpedia.org/page/'+p_df['Condition'][l])
                subj = URIRef('http://dbpedia.org/page/'+pst)
                sdg.add( (robj, RDF.type, disease) )
                sdg.add( (subj, RDF.type, anat) )
                sdg.add((robj, anatomicOn, subj))
                sdg.add((robj, probability, Literal('unwellPart-'+pst+':'+str(1/c))))
        l=l+1
    k=0
    agecount = {}
    for age in p_df['Group']:
        avsym = 0
        for a in age:
            for agi in agegroup:
                if agi == a:
                    avsym = avsym+1
            for sexi in sex:
                if sexi == a:
                    avsym = avsym+1
            agecount[a] = avsym
        for a in age:
            #print (a,agegroup)
            if a in agegroup:
                c = agecount[a]
                #if(p_df['Condition'][k]=='urinary-incontinence'):
                    #print(a, p_df['Condition'][k])
                if p_df['Condition'][k] in scoredict:
                    x = scoredict.get(p_df['Condition'][k])
                    scoredict[p_df['Condition'][k]]=x+1/c
                    y = matchdict.get(p_df['Condition'][k])
                    matchdict[p_df['Condition'][k]]=y+1
                else:
                    scoredict[p_df['Condition'][k]]=1/c
                    matchdict[p_df['Condition'][k]]=1
                robj = URIRef('http://dbpedia.org/page/'+p_df['Condition'][k])
                subj = URIRef('http://dbpedia.org/page/'+a)
                sdg.add( (robj, RDF.type, disease) )
                sdg.add( (subj, RDF.type, group) )
                sdg.add((robj, agegroup, subj))
                sdg.add((robj, probability, Literal('ageGroup-'+a+':'+str(1/c))))
            if a in sex:
                c = agecount[a]
                #if(p_df['Condition'][k]=='urinary-incontinence'):
                    #print(a, p_df['Condition'][k])
                if p_df['Condition'][k] in scoredict:
                    x = scoredict.get(p_df['Condition'][k])
                    scoredict[p_df['Condition'][k]]=x+1/c
                    y = matchdict.get(p_df['Condition'][k])
                    matchdict[p_df['Condition'][k]]=y+1
                else:
                    scoredict[p_df['Condition'][k]]=1/c
                    matchdict[p_df['Condition'][k]]=1
                robj = URIRef('http://dbpedia.org/page/'+p_df['Condition'][k])
                subj = URIRef('http://dbpedia.org/page/'+a)
                sdg.add( (robj, RDF.type, disease) )
                sdg.add( (subj, RDF.type, group) )
                sdg.add((robj, agegroup, subj))
                sdg.add((robj, probability, Literal('ageGroup-'+a+':'+str(1/c))))
        k=k+1
    normlisedict={}
    for x, y in scoredict.items():
        normlisedict[x]=y/5
        robj = URIRef('http://dbpedia.org/page/'+x)
        sdg.add((robj, score, Literal(str(y/5))))
    sdg.serialize(destination=filepath, format='turtle')
    sdg.close()
    ordereddict= {k: v for k, v in sorted(normlisedict.items(), key=lambda item: item[1], reverse=True)}
    return ordereddict

In [451]:
#testing example: 0 is diabetes testing dataset, 1 is allergy testing dataset, 2 is asthma testing dataset ... 14 is glaucoma 	
#we genereted 10 different version of 15 testing datasets and average accurate rate on 91%

In [452]:
i=4
s_inp=p_df_test.iloc[i]['Symptom']
d_inp=p_df_test.iloc[i]['Disease']
p_inp=p_df_test.iloc[i]['Position']
gr_inp=p_df_test.iloc[i]['Group']
sex_inp=p_df_test.iloc[i]['Sex']

In [453]:
ranking = predict_ranking(s_inp,d_inp,p_inp,gr_inp,sex_inp)

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> .  does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http:/

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> .  does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/resource/Category:Symptoms_and_signs does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/0

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/resource/Category:Symptoms_and_signs does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/resource/Category:Symptoms_and_signs> . } does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/page/Therapy does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/page/Therapy does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/page/Therapy> . } does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?r WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?r WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < do

...

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/page/Category:Human_development does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/page/Category:Human_development does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/page/Category:Human_development> . } does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT D

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> .  does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http:/

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> .  does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/resource/Category:Symptoms_and_signs does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/0

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/resource/Category:Symptoms_and_signs does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/resource/Category:Symptoms_and_signs> . } does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m

In [454]:
ranking

{'chickenpox': 0.8999999999999998,
 'pneumonia': 0.4666666666666667,
 'allergy': 0.4,
 'flu': 0.4,
 'breast-cancer': 0.3,
 'heart-attack': 0.3,
 'stroke': 0.3,
 'gallstone': 0.2,
 'urinary-tract-infection': 0.2,
 'asthma': 0.1,
 'lung-cancer': 0.1,
 'urinary-incontinence': 0.1,
 'glaucoma': 0.1}